In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.retrievers import BM25Retriever
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders.parsers import RapidOCRBlobParser
from langchain.retrievers import EnsembleRetriever

In [13]:
pdf_path="22365_3_Prompt Engineering_v7 (1).pdf"


In [14]:
#  Initialize the loader with image extraction enabled
loader = PyPDFLoader(
    file_path=pdf_path,
    extract_images=False,
    images_parser=RapidOCRBlobParser()
)

#  Load the documents (this processes both text and image-based content)
docs = loader.load()

#  Display the first few lines and metadata of the first page
print(" Extracted content from page 1:")
print(docs[0].page_content)  # Show first 500 characters
print("\n Metadata:")
print(docs[0].metadata)

 Extracted content from page 1:
Prompt  
Engineering
Author: Lee Boonstra

 Metadata:
{'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 20.2 (Macintosh)', 'creationdate': '2025-03-17T13:40:21-06:00', 'moddate': '2025-03-17T13:40:26-06:00', 'trapped': '/False', 'source': '22365_3_Prompt Engineering_v7 (1).pdf', 'total_pages': 68, 'page': 0, 'page_label': '1'}


In [15]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,  # %10-20 of chunk size. It Provides strong contextual chunks based on overlapping number.
    separators=["\n\n", "\n", ".", " ", ""]  # paragraph,newline,sentence,word..
)

# Split documents
chunked_documents = text_splitter.split_documents(docs)
documents = chunked_documents

In [16]:
print(f"Split {len(documents)} documents into {len(chunked_documents)} chunks")

Split 123 documents into 123 chunks


In [17]:
import os
from dotenv import load_dotenv
load_dotenv()
gem_api_key=os.getenv("GEMINI_API_KEY") # Get Api key


In [18]:
bm25=BM25Retriever.from_documents(documents,k=5) #Bm25 retriever(keyword search)
vectorstore=FAISS.from_documents(documents=documents,embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=gem_api_key)) # semantic  retriever
semantic_retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":5}) 
hybrid_retriever=EnsembleRetriever(retrievers=[bm25,semantic_retriever],weights=[0.5,0.5]) #Hybrid approach

In [19]:
query="system , contextual and role prompting explain"
bm25_results=bm25.invoke(input=query)     # Bm25 results
print(f"Found {len(bm25_results) }  documents")
for i,doc in enumerate(bm25_results):
    print(f"Rank {i+1}:")
    print(doc.page_content)
    print("-" * 10)

Found 5  documents
Rank 1:
Prompt Engineering
February 2025
18
System, contextual and role prompting
System, contextual and role prompting are all techniques used to guide how LLMs generate 
text, but they focus on different aspects:
• System prompting sets the overall context and purpose for the language model. It 
defines the ‘big picture’ of what the model should be doing, like translating a language, 
classifying a review etc.
• Contextual prompting provides specific details or background information relevant to 
the current conversation or task. It helps the model to understand the nuances of what’s 
being asked and tailor the response accordingly.
• Role prompting assigns a specific character or identity for the language model to adopt. 
This helps the model generate responses that are consistent with the assigned role and its 
associated knowledge and behavior.
There can be considerable overlap between system, contextual, and role prompting. E.g. a
----------
Rank 2:
Introductio

In [20]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=gem_api_key) 


# This chain will take the query, use the retriever to get context,
# and then pass the context and query to the LLM to generate the answer.
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff", # 'stuff' puts all documents into the prompt
    retriever=hybrid_retriever,
    return_source_documents=True #  return the docs used to answer
)

#  Run the chain with  query
print(f"\n Question: {query}")
response = qa_chain.invoke({"query": query})


print("\n Answer:")
print(response["result"])

if "source_documents" in response:
    print("\n Source Documents:")
    for i, doc in enumerate(response["source_documents"]):
        print(f"--- Source {i+1} (Page {doc.metadata.get('page_label', 'N/A')}) ---")
        print(doc.page_content[:300] + "...") # Show beginning of the source chunk
        print(f"Source: {doc.metadata.get('source', 'N/A')}, Page: {doc.metadata.get('page', 'N/A')}") # More detailed source info



 Question: system , contextual and role prompting explain

 Answer:
System prompting sets the overall context and purpose for the language model.  It defines the big picture of what the model should be doing (e.g., translating a language, classifying a review).

Contextual prompting provides specific details or background information relevant to the current conversation or task.  It helps the model understand the nuances of what's being asked and tailor the response accordingly.

Role prompting assigns a specific character or identity for the language model to adopt. This helps the model generate responses consistent with the assigned role and its associated knowledge and behavior.  There can be overlap between these three types of prompting.

 Source Documents:
--- Source 1 (Page 18) ---
Prompt Engineering
February 2025
18
System, contextual and role prompting
System, contextual and role prompting are all techniques used to guide how LLMs generate 
text, but they focus on different a